### Imports

In [1]:
import numpy as np
import pandas as pd
import time
from functools import partial

from scipy.stats import randint, uniform, beta
from sim_lib import simulation

pd.options.mode.chained_assignment = None

In [9]:
def policy_evaluation(policy, verbose=True):
    seed = 18475
    np.random.seed(seed=seed)
    start = time.time()
    print("Starting to evaluate policy...")
    output = simulation(policy, n=200_000, seed=seed, verbose=verbose)
    end = time.time()
    print(f"Evaluation is done! Total time: {end - start:.2f} seconds\n")
    return {
        "regret": output["regret"],
        "mean_regret": output["regret"] / output["rounds"],
        "total_banners": output["total_banners"],
    }

### Baseline

Первым делом посмотрим на бейзлайн алгоритм - `e-greedy` политику

In [2]:
def eps_greedy(history: pd.DataFrame, eps: float):
    if uniform.rvs() < eps:
        n = history.shape[0]
        return history.index[randint.rvs(0, n)]

    ctr = history["clicks"] / (history["impressions"] + 10)
    n = np.argmax(ctr)
    return history.index[n]


eps_greedy_policy = partial(eps_greedy, eps=0.06)

In [6]:
print(f"Regret is {policy_evaluation(eps_greedy_policy, verbose=False)['regret']}")

Starting to evaluate policy...


Evaluation is done! Total time: 575.22 seconds
Regret is 1540.7609683932544


### My policy - UCB

В качестве политики я выбрала `UCB`

In [4]:
class UCB:
    def __init__(self, C):
        self.C = C
        self.calls = 0

    def __call__(self, history: pd.DataFrame):
        self.calls += 1
        ctr = history["clicks"] / (history["impressions"] + 1)
        exploration = np.sqrt(2 * np.log(self.calls) / (history.impressions + 1))
        expectation = ctr + self.C * exploration
        n = np.argmax(expectation)
        return history.index[n]

Давайте сначала проверим, как работает наш алгоритм с `C = 1`

In [5]:
print(f"Regret is {policy_evaluation(UCB(C=1), verbose=False)['regret']}")

Starting to evaluate policy...


Evaluation is done! Total time: 531.22 seconds
Regret is 11185.91629936366


*Результат не очень хороший, поэтому будем перебирать параметр `exploration`*

In [10]:
C = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5]

In [11]:
results = {}
for c in C:
    print(f"C = {c}")
    results[c] = policy_evaluation(UCB(C=c), verbose=False)["regret"]

C = 0.0001
Starting to evaluate policy...
Evaluation is done! Total time: 531.73 seconds

C = 0.0005
Starting to evaluate policy...
Evaluation is done! Total time: 550.57 seconds

C = 0.001
Starting to evaluate policy...
Evaluation is done! Total time: 543.02 seconds

C = 0.005
Starting to evaluate policy...
Evaluation is done! Total time: 553.38 seconds

C = 0.01
Starting to evaluate policy...
Evaluation is done! Total time: 519.12 seconds

C = 0.05
Starting to evaluate policy...
Evaluation is done! Total time: 543.29 seconds

C = 0.1
Starting to evaluate policy...
Evaluation is done! Total time: 515.78 seconds

C = 0.5
Starting to evaluate policy...
Evaluation is done! Total time: 506.40 seconds



In [12]:
pd.DataFrame.from_dict(results, orient="index", columns=["regret"])

,regret
0.0001,7570.250349
0.0005,7570.250349
0.0010,7570.250349
0.0050,7570.250349
0.0100,974.101387
0.0500,2424.896208
0.1000,258.315825
0.5000,4013.025866


Результаты интересные, для значений `C < 0.005` получились одинаковые значения регрета, лучший результат для `C = 0.1`. Не наблюдается какой-то однозначной зависимости между коэффициентом эксплорейшена и регретом.
Возможно, слишком маленькие коэффициенты приводят к тому, что часто дергаем ручки жадно, почти не исследуем и хорошее приближение для награды получить не удается. Оптимальный эксплорейшен нужно прям поискать) Предлагается посмотреть на значения в окрестности `0.1`, поскольку с этим коэффициентом получился хороший результат.

In [13]:
C = [0.08, 0.09, 0.2, 0.3]

In [14]:
results = {}
for c in C:
    print(f"C = {c}")
    results[c] = policy_evaluation(UCB(C=c), verbose=False)["regret"]

C = 0.08
Starting to evaluate policy...
Evaluation is done! Total time: 512.52 seconds

C = 0.09
Starting to evaluate policy...
Evaluation is done! Total time: 522.49 seconds

C = 0.2
Starting to evaluate policy...
Evaluation is done! Total time: 563.35 seconds

C = 0.3
Starting to evaluate policy...
Evaluation is done! Total time: 537.36 seconds



In [15]:
pd.DataFrame.from_dict(results, orient="index", columns=["regret"])

,regret
0.08,197.726862
0.09,228.795585
0.20,797.937764
0.30,1665.415854


Получилось улучшить результат: `regret = 198` при `C = 0.08`